In [2]:
# Input a age boundary, such as 253
# The notebook will first find the latest scheme ID of the golobal geo time scale 
# Then to return a concept list (in which the input age boundary locates) from that vocabulary scheme

install.packages("SPARQL")
library(SPARQL)

Installing package into ‘/nethome/max/.R-libs’
(as ‘lib’ is unspecified)

Loading required package: XML

Loading required package: RCurl



In [3]:
# set up end point and prefix
endpoint = "http://virtuoso.nkn.uidaho.edu:8890/sparql/"
  
# attach SPARQL querry prefix. Note: the graph for our study should be updated
sparql_prefix = " 
                prefix dc: <http://purl.org/dc/elements/1.1/>
                prefix gts: <http://resource.geosciml.org/ontology/timescale/gts#>
                prefix skos: <http://www.w3.org/2004/02/skos/core#>
                prefix time: <http://www.w3.org/2006/time#>
                prefix ts: <http://resource.geosciml.org/vocabulary/timescale/> 
                prefix isc: <http://resource.geosciml.org/classifier/ics/ischart/> 
        "

In [4]:
# the query code
 
q = paste0(
              sparql_prefix, 
                  '
                   SELECT DISTINCT ?schURI ?lbl
                   WHERE
                   {
                     GRAPH <http://deeptimekb.org/iscallnew> 
                     {
                        ?schURI a skos:ConceptScheme ;
                                rdfs:label ?lbl .
                        FILTER regex(?lbl, "International", "i")
                     }      
                   }
                   ORDER BY DESC (?schURI)
                   LIMIT 1
                  '
              )

In [5]:
# run the query  
resScheme = SPARQL(endpoint, q)$results
  
return(resScheme)

schURI,lbl
<chr>,<chr>
<http://resource.geosciml.org/vocabulary/timescale/isc2018-08>,"""International Chronostratigraphic Chart (2018)""@en"


In [11]:
# This query will find the Age-level concept
# The scheme ID found in the last step will be used
    
q0 = paste0( sparql_prefix, '
SELECT DISTINCT ?tconcept  ?lbl
WHERE
{
  GRAPH <http://deeptimekb.org/iscallnew>
 {
    ?tconcept  a  gts:Age ; 
               skos:inScheme ', resScheme$schURI[1], '; 
               rdfs:label ?lbl;
               dc:description
                   [ 
                       time:hasBeginning ?basePosition ;
                       time:hasEnd ?topPosition ;
                       skos:inScheme ', resScheme$schURI[1], '
                   ] . 
       ?basePosition time:inTemporalPosition ?baseTime .
       ?topPosition time:inTemporalPosition ?topTime .

       ?baseTime   dc:description
                   [ 
                      time:numericPosition ?baseNum ;
                      skos:inScheme ', resScheme$schURI[1], '
                   ] .
       ?topTime   dc:description
                   [ 
                      time:numericPosition ?topNum ;
                      skos:inScheme ', resScheme$schURI[1], '
                   ] .

    FILTER (?baseNum >= 253)
    FILTER (?topNum <= 253) 
  }
}
ORDER BY ?baseNum
LIMIT 1
 '
     )
res0 = SPARQL(endpoint, q0)$results

In [18]:
# This query will find and list all the concepts at different levels
# We avoid using nested query to save time
    
q = paste0(
              sparql_prefix, 
                  '
SELECT DISTINCT ?All ?cLabelEn
WHERE
{
GRAPH <http://deeptimekb.org/iscallnew>
{

{ #This block is to include the Age-level concept in the resulting list 
  ', res0$tconcept[1], ' rdfs:label ?labelAgeC . 
  ?All rdfs:label ?labelAgeC .
  FILTER (?All = ', res0$tconcept[1], ') #only use label is not enough to make sure the two concepts are the same
}
UNION
{ # This and the next block are to find the broader and broaderTransitive concepts
', res0$tconcept[1], '
dc:description
      [
         skos:broaderTransitive ?All ;
         skos:inScheme  ts:isc2012-08
      ]
}
UNION 
{
', res0$tconcept[1], '
dc:description
      [
         skos:broader  ?All ;
         skos:inScheme  ts:isc2012-08
      ]

}
  
      ?All 
      rdfs:label ?cLabelEn ;
      dc:description 
      [  
        time:hasBeginning ?baseboundary1 ;
        time:hasEnd ?topboundary1 ;
        skos:inScheme ts:isc2012-08
        ] .
      
      ?baseboundary1 time:inTemporalPosition ?basetime1 .
      ?topboundary1 time:inTemporalPosition ?toptime1 .
      
      ?basetime1 dc:description
      [
        time:numericPosition ?baseNum1;
        skos:inScheme ts:isc2012-08
        ]  .
      
      ?toptime1 dc:description
      [
        time:numericPosition ?topNum1;
        skos:inScheme ts:isc2012-08
        ]  .
      
    }
}

ORDER BY (?baseNum1 - ?topNum1)
                  '
              )
# run the query  
res = SPARQL(endpoint, q)$results
  
print(res)

                                                                  All
1 <http://resource.geosciml.org/classifier/ics/ischart/Changhsingian>
2     <http://resource.geosciml.org/classifier/ics/ischart/Lopingian>
3       <http://resource.geosciml.org/classifier/ics/ischart/Permian>
4     <http://resource.geosciml.org/classifier/ics/ischart/Paleozoic>
5   <http://resource.geosciml.org/classifier/ics/ischart/Phanerozoic>
                cLabelEn
1 "Changhsingian Age"@en
2   "Lopingian Epoch"@en
3    "Permian Period"@en
4     "Paleozoic Era"@en
5   "Phanerozoic Eon"@en


In [14]:
#Below is another solution to find the list of concepts at different levels 
ConceptLevels <- c("Age", "Epoch", "Sub-Period", "Period", "Era", "Eon", "Super-Eon")

cLevelLength = length(ConceptLevels)

cat("In ts:isc2012-08","\n")

for (j in 1:cLevelLength)  {  
q = paste0(
              sparql_prefix, 
                  '
SELECT DISTINCT ?tconcept  str(?lbl) AS ?cLabel
WHERE
{
    {
       ?tconcept  a gts:GeochronologicEra ,
                 gts:', ConceptLevels[j] , ';  
               skos:inScheme ts:isc2012-08 ; 
               rdfs:label ?lbl ;
               time:hasBeginning ?basePosition ;
               time:hasEnd ?topPosition. 
      ?basePosition time:inTemporalPosition ?baseTime .
      ?topPosition time:inTemporalPosition ?topTime .

      ?baseTime time:numericPosition ?baseNum .
      ?topTime time:numericPosition ?topNum .
    }
   UNION 
    { 
       ?tconcept  a gts:GeochronologicEra ,
                 gts:', ConceptLevels[j] , ';  
               skos:inScheme ts:isc2012-08; 
               rdfs:label ?lbl ;
               dc:description
                   [ 
                       time:hasBeginning ?basePosition ;
                       time:hasEnd ?topPosition ;
                       skos:inScheme ts:isc2012-08
                   ] . 
       ?basePosition time:inTemporalPosition ?baseTime .
       ?topPosition time:inTemporalPosition ?topTime .

       ?baseTime   dc:description
                   [ 
                      time:numericPosition ?baseNum ;
                      skos:inScheme ts:isc2012-08
                   ] .
       ?topTime   dc:description
                   [ 
                      time:numericPosition ?topNum ;
                      skos:inScheme ts:isc2012-08
                   ] .
     } 

    FILTER (?baseNum >= 253)
    FILTER (?topNum <= 253) 

}
ORDER BY ?baseNum
LIMIT 1
        '
          )
# run the query  
resConcepts = SPARQL(endpoint, q)$results

cat(ConceptLevels[j], "Level: ", resConcepts$cLabel[1], "\n")
    
}

In ts:isc2012-08 
Age Level:  Changhsingian Age 
Epoch Level:  Lopingian Epoch 
Sub-Period Level:  
Period Level:  Permian Period 
Era Level:  Paleozoic Era 
Eon Level:  Phanerozoic Eon 
Super-Eon Level:  
